# Q Learning 
<b> A Q table based Q-learning implementation for a game from the OpenAI library</b>

This notebook serves mostly as a reference for myself. This is my first implementation of RL. Hence I started with the Q-table implementation. In another notebook I will repeat this excercise using Deep-Q-learning. 

The game we will be using in this notebook are _FrozenLake-v0_ and _FrozenLake8x8-v0_.

Here is the premise:
```
Winter is here. You and your friends were tossing around a frisbee at the park when you made a wild throw that left the frisbee out in the middle of the lake. The water is mostly frozen, but there are a few holes where the ice has melted. If you step into one of those holes, you'll fall into the freezing water. At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend.

The surface is described using a grid like the following:

SFFF       (S: starting point, safe)
FHFH       (F: frozen surface, safe)
FFFH       (H: hole, fall to your doom)
HFFG       (G: goal, where the frisbee is located)

The episode ends when you reach the goal or fall in a hole. You receive a reward of 1 if you reach the goal, and zero otherwise.
```
[Source](https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py)

In [1]:
import numpy as np
import gym
import random

In [2]:
env = gym.make("FrozenLake8x8-v0")

In [8]:
print(env.action_space)
print(env.observation_space)

Discrete(4)
Discrete(64)


The game has 4 different actions one can take

- LEFT = 0
- DOWN = 1
- RIGHT = 2
- UP = 3


In [9]:
env.render()


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


So that is how the grid looks like. 

In [14]:
action_state = env.action_space.n  
state_size = env.observation_space.n

We will start with an empty Q-table, where the columns are the possible actions and each row corresponds to the state of the game

In [18]:
qtable = np.zeros((state_size, action_size))
qtable.shape

(64, 4)

The Reinforcement learning algorithm in pseudo-code looks like this:

```
1. Intialize Q-values (Q(s,a)) arbitrarily for all state-action pairs
2. Until a stopping criteria is met do:
3. Choose an action (a) in the current world state (s) based on current Q-value estimates.
4. Take the action (a) and observe the outcome state (s') and reward (r)
5. Update the Q-table -> Q(s,a) := Q(s,a) + alpha * (r + gamma * max(Q(s',a')) - Q(s, a))
```

[Source](https://medium.freecodecamp.org/diving-deeper-into-reinforcement-learning-with-q-learning-c18d0db58efe)

Some playing around here to see how the game works and what the output means

In [76]:
env.reset()
env.render()
env.step(1)
env.render()


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


In [142]:
class Agent:
    def __init__(self, environment, learning_rate, discount_rate):
        self.env = environment
        self.alpha = learning_rate
        self.gamma = discount_rate
        self.q_table = np.zeros((env.observation_space.n, env.action_space.n ))
    
    def train(self, total_episodes, epsilon, max_epsilon, min_epsilon, decay_rate):
        rewards = []
        for episode in range(total_episodes):
            state = env.reset()
            step = 0
            done = False
            total_rewards = 0
#             print('in epsode', episode, 'eps is', epsilon)
            while True:
                # choose an action based on Q table (step 3)
                # here we implement the exploration possibility in the beginning we want to take random actions
                # so the agent can explore the environment, later once it has learned about the environment it can 
                # start to take actions it knows
                if np.random.uniform(high=1, low=0) < epsilon:
                    action = env.action_space.sample()
                else:
                    action = np.argmax(self.q_table[state])
                    
                # take the action and observe the outcome state and reward (step 4)
             
                next_state, reward, done, info = env.step(action)
                
                 
                # update q table (step 5)
                self.q_table[state, action] = self.q_table[state, action] + self.alpha * (
                reward + self.gamma * np.max(self.q_table[next_state]) - self.q_table[state, action])
                
                state = next_state
                total_rewards += reward

                if done == True:
                    break    

            #adjust epsilon by an exponential function with lower bound min_epsilon
            epsilon = min_epsilon + (max_epsilon-min_epsilon)*np.exp(-decay_rate*episode) 
            rewards.append(total_rewards)    
        print ("Score over time: " +  str(sum(rewards)/total_episodes))
    
    def play(self, episodes, max_steps = 1000, render=False):
       
        for episode in range(episodes):
            state = env.reset()
            step = 0
            done = False
            score = 0
            print("****************************************************")
            print("EPISODE ", episode)
            while score != 1:
                if render:
                    env.render()
                else:
                    print('agent in pos', state)
                # take the action with the maximum expected reward
                action = np.argmax(self.q_table[state])
                next_state, reward, done, info = env.step(action)
                step += 1
                score += reward
                if done and reward == 0:
                    print('agent died in step', step, ' traveling from state', state, 'to', next_state)
                    next_state = env.reset()
                    step = 0
                elif reward == 1:
                    print('agent reached finish in step', step, ' traveling from state', state, 'to', next_state)
                state = next_state
        

In [168]:
# Parameters
nr_of_episodes = 1000 
alpha = 0.75 # learning rate
gamma = 0.95 #discount rate (i.e. how much of future actions do we need to take into consideration)

#The exploration/exploitation parameters
epsilon = 1
max_epsilon = 1
min_epsilon = 0.01
decay_rate = 0.01

env = gym.make("FrozenLake-v0")
# env = gym.make("FrozenLake8x8-v0")
agent = Agent(env, alpha, gamma )
agent.train(nr_of_episodes, epsilon, max_epsilon, min_epsilon, decay_rate)

Score over time: 0.333


In [125]:
agent.play(1, render = True)

****************************************************
EPISODE  0

SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  

In [170]:
# Parameters
nr_of_episodes = 3000 
alpha = 0.75 # learning rate
gamma = 0.95 #discount rate (i.e. how much of future actions do we need to take into consideration)

#The exploration/exploitation parameters
epsilon = 1
max_epsilon = 1
min_epsilon = 0.01
decay_rate = 0.00001

# env = gym.make("FrozenLake-v0")
env = gym.make("FrozenLake8x8-v0")
agent = Agent(env, alpha, gamma )
agent.train(nr_of_episodes, epsilon, max_epsilon, min_epsilon, decay_rate)

Score over time: 0.002


Note that it is important to reduce the decay rate for this environment, because compared with the 4x4 grid more exploration is required. With a low decay rate this becomes impossible and the agent is not trained well. 

In [172]:
agent.play(1, render=True)

****************************************************
EPISODE  0

SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
agent died in step 11  traveling from state 18 to 19


  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
F

There is randomness in the game, so let's play again

In [173]:
agent.play(1, render=True)

****************************************************
EPISODE  0

SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
agent died in step 7  traveling from state 11 to 19

SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SF

In [175]:
agent.play(1, render=False)

****************************************************
EPISODE  0
agent in pos 0
agent in pos 0
agent in pos 0
agent in pos 8
agent in pos 9
agent in pos 1
agent in pos 9
agent in pos 17
agent in pos 9
agent in pos 1
agent in pos 2
agent in pos 3
agent in pos 11
agent died in step 13  traveling from state 11 to 19
agent in pos 0
agent in pos 1
agent in pos 0
agent in pos 1
agent in pos 2
agent in pos 1
agent in pos 2
agent in pos 2
agent in pos 1
agent in pos 0
agent in pos 1
agent in pos 2
agent in pos 3
agent in pos 2
agent in pos 3
agent in pos 4
agent in pos 3
agent in pos 11
agent died in step 18  traveling from state 11 to 19
agent in pos 0
agent in pos 8
agent in pos 9
agent in pos 1
agent in pos 0
agent in pos 0
agent in pos 0
agent in pos 8
agent in pos 8
agent in pos 8
agent in pos 0
agent in pos 0
agent in pos 1
agent in pos 2
agent in pos 2
agent in pos 1
agent in pos 0
agent in pos 0
agent in pos 0
agent in pos 8
agent in pos 8
agent in pos 0
agent in pos 0
agent in pos 1
ag

In [176]:
agent.play(1, render=False)

****************************************************
EPISODE  0
agent in pos 0
agent in pos 0
agent in pos 1
agent in pos 9
agent in pos 17
agent in pos 16
agent in pos 16
agent in pos 8
agent in pos 0
agent in pos 1
agent in pos 2
agent in pos 2
agent in pos 1
agent in pos 9
agent in pos 17
agent in pos 16
agent in pos 17
agent in pos 9
agent in pos 17
agent in pos 16
agent in pos 8
agent in pos 0
agent in pos 1
agent in pos 2
agent in pos 2
agent in pos 2
agent in pos 2
agent in pos 1
agent in pos 0
agent in pos 1
agent in pos 0
agent in pos 8
agent in pos 0
agent in pos 1
agent in pos 0
agent in pos 1
agent in pos 2
agent in pos 1
agent in pos 2
agent in pos 3
agent in pos 4
agent in pos 3
agent in pos 2
agent in pos 1
agent in pos 0
agent in pos 8
agent in pos 9
agent in pos 10
agent in pos 2
agent in pos 2
agent in pos 1
agent in pos 0
agent in pos 0
agent in pos 8
agent in pos 9
agent in pos 10
agent in pos 11
agent in pos 12
agent in pos 4
agent in pos 12
agent in pos 20
agent i

Due to the randomness in the game, the agent dies a few times, but is able to eventually finish the game in a not too many steps. 

In [ ]:
L